In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-23


In [3]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH_OHLCV = f'..\data\{date_str}_df_OHLCV_clean.parquet'
SOURCE_PATH = f'..\data\{date_str}_df_finviz_merged.parquet'
PARQUET_PATH_COVARIANCE = f'..\data\{date_str}_df_cov_emv_matrix.parquet'
PARQUET_PATH_CORRELATION = f'..\data\{date_str}_df_corr_emv_matrix.parquet'

Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


In [4]:
import pandas as pd

df_ohlcv = pd.read_parquet(SOURCE_PATH_OHLCV)
df_finviz = pd.read_parquet(SOURCE_PATH)

print(f"Loaded df_ohlcv shape: {df_ohlcv.shape}")
print(f"Loaded df_finviz shape: {df_finviz.shape}")


Loaded df_ohlcv shape: (384250, 9)
Loaded df_finviz shape: (1537, 55)


In [5]:
# Combine indices from both dataframes and remove duplicates
tickers = list(set(df_finviz.index))
print(f"Total unique tickers: {len(tickers)}")

Total unique tickers: 1537


In [6]:
df_close = df_ohlcv['Adj Close'].unstack(level=0)

# Assume `tickers` is your predefined list of tickers
common_tickers = df_close.columns.intersection(tickers)
df_close = df_close[common_tickers]

# Assuming `tickers` is your original list of tickers
missing_tickers = list(set(tickers) - set(df_close.columns))

# Print the missing tickers (if any)
if missing_tickers:
    print(f"{len(missing_tickers)} tickers not found in df_close:")
    print(missing_tickers)
else:
    print("All tickers in your list are present in df_close.")

print(f"\nShape of filtered df_close: {df_close.shape}")
display(df_close.head())

All tickers in your list are present in df_close.

Shape of filtered df_close: (250, 1537)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABEV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG   AIRR     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES   ARKB   ARKK     ARM   ARMK     ARW     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
2024-04-24  136.47  35.67  13.92  89.33  168.23  161.92  2.2982  162.84  104.79  88.61  19.54  33.33  93.08  310.91  104.91  100.45  50.69  477.12  55.89  193.29  59.31  241.42  215.00  6.19  72.37  5.77  63.01  83.17  84.43  16.59  121.16  82.56  32.08  114.64  92.14  15.16  8.08  73.40  64.06  184.81  174.39  234.85  102.19  70.99  113.23  79.31  313.78  44.23  168.87  125.10  38.56  143.80  79.15  116.51  13.41  194.42  8.63  151.74  177.06  264.93  34.54  43.89  407.56  167.80  17.77  176.59  159.52  63.69  328.57  306.89  85.52  30.65  228.87  37.51  57.63  111.17  69.70  215.90  71.40  31.44  18.98  113.87  132.44  63.91  43.90   99.88  32.20  127.58  14.33  886.76  143.18  327.99  2.14  10.08  49.22  115.74  138.80  21.30  2.84  185.34  60.32  61.86  56.03  124.16  61.12  25.46  85.13  88.73  208.83  114.94  118.61  302.41  236.45  61.27  31.11  252.50  45.40  69.77  2988.51  164.33  73.04  37.40  142.26  64.34  37.89  164.60  39.87  45.06  63.60  2.4042  56.14  56.61  25.03   
2024-04-25  135.36  35.54  14.13  89.56  169.09  161.43  2.2784  163.01  104.76  88.48  19.51  32.77  92.67  306.40  104.58  100.34  50.64  473.44  55.58  194.71  58.77  241.16  216.40  6.14  72.41  5.78  64.25  83.64  84.50  16.36  120.22  82.02  31.65  113.66  91.88  14.94  7.97  73.06  64.04  177.71  171.03  234.99  101.79  71.59  112.95  78.58  310.50  44.55  168.94  123.45  37.96  143.71  79.38  114.44  13.61  195.85  8.53  153.76  176.77  261.41  34.73  43.76  405.64  167.36  17.76  173.67  160.38  66.19  327.04  304.08  81.41  30.94  229.26  37.68  58.97  110.34  69.21  216.65  70.13  33.39  18.84  111.63  130.28  64.76  43.61   97.93  31.98  127.81  14.55  896.89  136.93  335.25  2.19   9.86  49.84  115.26  138.97  22.14  2.82  185.32  60.14  61.53  56.35  127.88  60.74  25.15  84.73  88.38  213.49  114.75  118.80  306.24  234.45  60.60  30.93  251.15  45.56  73.52  2945.25  166.81  73.51  37.00  143.13  64.60  37.69  164.14  39.19  45.00  63.81  2.3951  56.05  56.38  24.33   


In [7]:
df_close_sorted = df_close.sort_index()
print(f'df_close_sorted shape: {df_close_sorted.shape}')
display(df_close_sorted.head())

# Calculate daily returns
df_close_returns = df_close_sorted.pct_change().dropna()
print(f'df_close_returns shape: {df_close_returns.shape}')
display(df_close_returns.head())


df_close_sorted shape: (250, 1537)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABEV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG   AIRR     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES   ARKB   ARKK     ARM   ARMK     ARW     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
2024-04-24  136.47  35.67  13.92  89.33  168.23  161.92  2.2982  162.84  104.79  88.61  19.54  33.33  93.08  310.91  104.91  100.45  50.69  477.12  55.89  193.29  59.31  241.42  215.00  6.19  72.37  5.77  63.01  83.17  84.43  16.59  121.16  82.56  32.08  114.64  92.14  15.16  8.08  73.40  64.06  184.81  174.39  234.85  102.19  70.99  113.23  79.31  313.78  44.23  168.87  125.10  38.56  143.80  79.15  116.51  13.41  194.42  8.63  151.74  177.06  264.93  34.54  43.89  407.56  167.80  17.77  176.59  159.52  63.69  328.57  306.89  85.52  30.65  228.87  37.51  57.63  111.17  69.70  215.90  71.40  31.44  18.98  113.87  132.44  63.91  43.90   99.88  32.20  127.58  14.33  886.76  143.18  327.99  2.14  10.08  49.22  115.74  138.80  21.30  2.84  185.34  60.32  61.86  56.03  124.16  61.12  25.46  85.13  88.73  208.83  114.94  118.61  302.41  236.45  61.27  31.11  252.50  45.40  69.77  2988.51  164.33  73.04  37.40  142.26  64.34  37.89  164.60  39.87  45.06  63.60  2.4042  56.14  56.61  25.03   
2024-04-25  135.36  35.54  14.13  89.56  169.09  161.43  2.2784  163.01  104.76  88.48  19.51  32.77  92.67  306.40  104.58  100.34  50.64  473.44  55.58  194.71  58.77  241.16  216.40  6.14  72.41  5.78  64.25  83.64  84.50  16.36  120.22  82.02  31.65  113.66  91.88  14.94  7.97  73.06  64.04  177.71  171.03  234.99  101.79  71.59  112.95  78.58  310.50  44.55  168.94  123.45  37.96  143.71  79.38  114.44  13.61  195.85  8.53  153.76  176.77  261.41  34.73  43.76  405.64  167.36  17.76  173.67  160.38  66.19  327.04  304.08  81.41  30.94  229.26  37.68  58.97  110.34  69.21  216.65  70.13  33.39  18.84  111.63  130.28  64.76  43.61   97.93  31.98  127.81  14.55  896.89  136.93  335.25  2.19   9.86  49.84  115.26  138.97  22.14  2.82  185.32  60.14  61.53  56.35  127.88  60.74  25.15  84.73  88.38  213.49  114.75  118.80  306.24  234.45  60.60  30.93  251.15  45.56  73.52  2945.25  166.81  73.51  37.00  143.13  64.60  37.69  164.14  39.19  45.00  63.81  2.3951  56.05  56.38  24.33   


df_close_returns shape: (249, 1537)


Symbol             A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG      AIRR       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARKB      ARKK       ARM      ARMK       ARW        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
2024-04-25 -0.008134 -0.003645  0.015086  0.002575  0.005112 -0.003026 -0.008615  0.001044 -0.000286 -0.001467 -0.001535 -0.016802 -0.004405 -0.014506 -0.003146 -0.001095 -0.000986 -0.007713 -0.005547  0.007346 -0.009105 -0.001077  0.006512 -0.008078  0.000553  0.001733  0.019679  0.005651  0.000829 -0.013864 -0.007758 -0.006541 -0.013404 -0.008548 -0.002822 -0.014512 -0.013614 -0.004632 -0.000312 -0.038418 -0.019267  0.000596 -0.003914  0.008452 -0.002473 -0.009204 -0.010453  0.007235  0.000415 -0.013189 -0.015560 -0.000626  0.002906 -0.017767  0.014914  0.007355 -0.011587  0.013312 -0.001638 -0.013287  0.005501 -0.002962 -0.004711 -0.002622 -0.000563 -0.016535  0.005391  0.039253 -0.004657 -0.009156 -0.048059  0.009462  0.001704  0.004532  0.023252 -0.007466 -0.007030  0.003474 -0.017787  0.062023 -0.007376 -0.019672 -0.016309  0.013300 -0.006606 -0.019523 -0.006832  0.001803  0.015352  0.011424 -0.043651  0.022135  0.023364 -0.021825  0.012597 -0.004147  0.001225  0.039437   
2024-04-26  0.010047  0.025886 -0.017693  0.013845 -0.003489 -0.045902  0.013035  0.007484  0.006300 -0.023056  0.011276  0.016173  0.004856 -0.003198  0.009371 -0.002292  0.007899  0.008702 -0.004678  0.020338 -0.014633 -0.013311  0.007070  0.016287 -0.014639  0.012111  0.008872 -0.018412  0.003550  0.004890 -0.010647 -0.001341  0.053081 -0.009062  0.002177  0.021419  0.010038 -0.001369  0.011243  0.010917 -0.004385 -0.012001 -0.001081  0.187317  0.016467 -0.000891 -0.004767 -0.013244 -0.013141 -0.008181  0.004215 -0.002783 -0.071932  0.055575 -0.002939  0.029768  0.001172  0.023673 -0.002319  0.002219  0.018140  0.003199 -0.002194 -0.006991  0.029279  0.034260  0.059234 -0.000151  0.020517 -0.068535  0.004668  0.004202  0.004275  0.010881  0.012549  0.002991  0.066609  0.120471  0.015400  0.012279  0.006900 -0.009048 -0.000844 -0.016059  0.010548  0.041050  0.000625  0.006025  0.033677  0.020359 -0.007522  0.019090  0.018265 -0.005071  0.038323 -0.010498  0.022451  0.046522   
2024-04-29  0.013458  0.

In [8]:
cov_emv_matrix, corr_emv_matrix = utils.get_cov_corr_ewm_matrices_chunked(df_close_returns, span=21, return_corr=True, return_cov=True)

print(f'cov_emv_matrix shape: {cov_emv_matrix.shape}')
display(cov_emv_matrix.head())
print(f'corr_emv_matrix shape: {corr_emv_matrix.shape}')
display(corr_emv_matrix.head())  

cov_emv_matrix shape: (1537, 1537)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG      AIRR       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARKB      ARKK       ARM      ARMK       ARW        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       0.000865  0.001277  0.001398  0.000925  0.001074  0.000504  0.000374  0.000910  0.000272  0.000458  0.000147  0.000786  0.000539  0.000597  0.000634  0.000294  0.000478  0.000558  0.000156  0.001201  0.000404  0.000490  0.000636  0.000518  0.000239  0.000724  0.000096  0.000151  0.000788  0.000865  0.000426  0.000465  0.001609  0.000819  0.000036  0.000143  0.000648  0.000483  0.000741  0.000885  0.000583  0.000410  0.000841  0.001431  0.001464  0.000601  0.000789  0.001398  0.000459  0.000455  0.000847  0.000905  0.000713  0.000712  0.000435  0.001124  0.000435  0.001460  0.000598  0.000406  0.000487  0.000500  0.000835  0.000112  0.000343  0.000935  0.000638  0.000884  0.000627  0.000402  0.000415  0.001557  0.000703  0.000737  0.000891  0.001158  0.001066  0.000725  0.000913  0.001123  0.000729  0.000698  0.001148  0.000562  0.001162  0.001500  0.000718  0.000704  0.001219  0.000960  0.000174  0.000524  0.001298  0.000948  0.001274  0.000178  0.000534  0.000174  0.001228   
AA      0.001277  0.002546  0.002183  0.001293  0.001684  0.000649  0.000545  0.001542  0.000354  0.000677  0.000193  0.001337  0.000810  0.000946  0.000988  0.000443  0.000768  0.000844  0.000204  0.001936  0.000523  0.000738  0.001076  0.000791  0.000284  0.001159  0.000323  0.000086  0.001066  0.001163  0.000652  0.000700  0.002552  0.001331  0.000054  0.000484  0.000897  0.000738  0.001154  0.001280  0.000921  0.000592  0.001391  0.002250  0.002391  0.000911  0.001326  0.002283  0.000654  0.000643  0.001408  0.001259  0.001111  0.001269  0.000537  0.001803  0.000629  0.002418  0.000960  0.000583  0.000748  0.000745  0.001303  0.000022  0.000310  0.001526  0.001007  0.001342  0.000967  0.000604  0.000620  0.002385  0.001072  0.001080  0.001417  0.001850  0.001736  0.001072  0.001386  0.001701  0.001103  0.000925  0.001873  0.001055  0.001921  0.002426  0.001153  0.001089  0.002055  0.001484  0.000098  0.000817  0.001662  0.001678  0.002044  0.000216  0.000893  0.000805  0.001815   


corr_emv_matrix shape: (1537, 1537)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG      AIRR       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARKB      ARKK       ARM      ARMK       ARW        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       1.000000  0.860487  0.798792  0.797620  0.867762  0.758108  0.583060  0.808881  0.556622  0.645460  0.178896  0.850094  0.811802  0.866558  0.898924  0.720113  0.807006  0.759882  0.332843  0.864277  0.573927  0.772110  0.813251  0.794156  0.507420  0.849722  0.123791  0.324958  0.870261  0.853373  0.649919  0.714538  0.817522  0.747655  0.302877  0.157175  0.831745  0.712621  0.877735  0.803749  0.771450  0.623145  0.850295  0.783356  0.884974  0.815415  0.785959  0.836096  0.623227  0.779285  0.808839  0.827112  0.831066  0.788307  0.644460  0.857534  0.705818  0.823195  0.895405  0.662122  0.753805  0.817434  0.858924  0.175860  0.486306  0.854764  0.732928  0.763955  0.849466  0.691685  0.707969  0.858273  0.882276  0.808805  0.802867  0.811782  0.527018  0.811113  0.828999  0.843301  0.853708  0.843067  0.818619  0.599884  0.875124  0.863091  0.790278  0.824097  0.869381  0.786076  0.216722  0.621977  0.703875  0.813627  0.818954  0.455648  0.819260  0.133865  0.679529   
AA      0.860487  1.000000  0.727150  0.649758  0.793611  0.569679  0.495522  0.799028  0.422901  0.556559  0.137116  0.843060  0.711520  0.800398  0.816389  0.632674  0.756069  0.670381  0.253347  0.812147  0.432990  0.678044  0.802518  0.706114  0.351673  0.792543  0.243445  0.107745  0.686317  0.668729  0.579881  0.626787  0.755875  0.708883  0.262927  0.309014  0.671120  0.635567  0.797229  0.677467  0.710063  0.525191  0.819415  0.718089  0.842595  0.720369  0.769383  0.795836  0.517097  0.642042  0.783880  0.670905  0.755108  0.819044  0.463853  0.802067  0.595595  0.794501  0.838190  0.554910  0.674816  0.709734  0.781638  0.020138  0.256057  0.813328  0.675097  0.676452  0.764458  0.605550  0.615767  0.766719  0.784145  0.691276  0.744549  0.756083  0.500284  0.698972  0.733747  0.745144  0.752421  0.652055  0.778403  0.657244  0.843101  0.813786  0.739903  0.743417  0.854731  0.708484  0.071575  0.565214  0.525281  0.839586  0.766291  0.322279  0.799466  0.361588  0.585371   


In [9]:
cov_emv_matrix.to_parquet(PARQUET_PATH_COVARIANCE, engine='pyarrow', compression='zstd')
print(f'cov_emv_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_emv_matrix.info())

corr_emv_matrix.to_parquet(PARQUET_PATH_CORRELATION, engine='pyarrow', compression='zstd')
print(f'corr_emv_matrix parqueted to {PARQUET_PATH_CORRELATION}')
display(corr_emv_matrix.info())

cov_emv_matrix parqueted to ..\data\2025-04-23_df_cov_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1537 entries, A to ZWS
Columns: 1537 entries, A to ZWS
dtypes: float64(1537)
memory usage: 18.1+ MB


None

corr_emv_matrix parqueted to ..\data\2025-04-23_df_corr_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1537 entries, A to ZWS
Columns: 1537 entries, A to ZWS
dtypes: float64(1537)
memory usage: 18.1+ MB


None